<a href="https://colab.research.google.com/github/thanadolch/data_viz/blob/main/kaggel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
import plotly.express as px
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '/content/drive/My Drive/DVZ'

In [4]:
df = pd.read_csv(os.path.join(path,'netflix_titles.csv'))
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
df['date_added'] =pd.to_datetime(df.date_added)
df = df.sort_values('date_added')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7113,s7114,Movie,To and From New York,Sorin Dan Mihalcescu,"Barbara King, Shaana Diya, John Krisiukenas, Y...",United States,2008-01-01,2006,TV-MA,81 min,"Dramas, Independent Movies, Thrillers","While covering a story in New York City, a Sea..."
1765,s1766,TV Show,Dinner for Five,NaN,NaN,United States,2008-02-04,2007,TV-MA,1 Season,Stand-Up Comedy & Talk Shows,"In each episode, four celebrities join host Jo..."
3248,s3249,Movie,Just Another Love Story,Ole Bornedal,"Anders W. Berthelsen, Rebecka Hemse, Nikolaj L...",Denmark,2009-05-05,2007,TV-MA,104 min,"Dramas, International Movies",When he causes a car accident that leaves a yo...
5765,s5766,Movie,Splatter,Joe Dante,"Corey Feldman, Tony Todd, Tara Leigh, Erin Way...",United States,2009-11-18,2009,TV-MA,29 min,Horror Movies,"After committing suicide, a washed-up rocker r..."
3840,s3841,Movie,Mad Ron's Prevues from Hell,Jim Monaco,"Nick Pawlow, Jordu Schell, Jay Kushwara, Micha...",United States,2010-11-01,1987,NR,84 min,"Cult Movies, Horror Movies","This collection cherry-picks trailers, forgott..."


In [6]:
def check_nan_values(dataset):
    for col in dataset:
        print("- {} = {}".format(col, df[col].isnull().sum()))
        
check_nan_values(df)

- show_id = 0
- type = 0
- title = 0
- director = 2389
- cast = 718
- country = 507
- date_added = 10
- release_year = 0
- rating = 7
- duration = 0
- listed_in = 0
- description = 0


In [7]:
del df['show_id']

In [8]:
df['rating'] = df['rating'].fillna(value=df['rating'].value_counts().idxmax())

In [9]:
df.dropna(subset=['date_added'],inplace=True)

In [10]:
df['director'] = df['director'].fillna("unknown")
df['cast'] = df['cast'].fillna("unknown")

In [11]:
check_nan_values(df)

- type = 0
- title = 0
- director = 0
- cast = 0
- country = 506
- date_added = 0
- release_year = 0
- rating = 0
- duration = 0
- listed_in = 0
- description = 0


In [12]:
df['country'].unique()[10:20]

array(['United Kingdom,', 'Canada, United States',
       'Norway, United States', 'United Kingdom, United States',
       'United States, Canada', nan,
       'Australia, United Kingdom, United States, New Zealand, Italy, France',
       'France', 'United Kingdom', 'Mexico'], dtype=object)

In [13]:
df.country = df.country.fillna("other")

In [14]:
def fix_country_col(data):
    new_col = []
    for row in data["country"]:
        new_col.append(row.split(","))
    return new_col

def fix_cast_col(data):
    new_col = []
    for row in data["cast"]:
        new_col.append(row.split(","))
    return new_col

def fix_listed_in_col(data):
    new_col = []
    for row in data["listed_in"]:
        new_col.append(row.lower().replace("&",",").replace("tv","").split(","))
    return new_col

df['country'] = fix_country_col(df)
df['listed_in'] = fix_listed_in_col(df)
df['cast'] = fix_cast_col(df)

In [15]:
df['date_added'] = [col.strftime('%Y') for col in df['date_added']]

In [16]:
df.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7113,Movie,To and From New York,Sorin Dan Mihalcescu,"[Barbara King, Shaana Diya, John Krisiukenas...",[United States],2008,2006,TV-MA,81 min,"[dramas, independent movies, thrillers]","While covering a story in New York City, a Sea..."
1765,TV Show,Dinner for Five,unknown,[unknown],[United States],2008,2007,TV-MA,1 Season,"[stand-up comedy , talk shows]","In each episode, four celebrities join host Jo..."
3248,Movie,Just Another Love Story,Ole Bornedal,"[Anders W. Berthelsen, Rebecka Hemse, Nikola...",[Denmark],2009,2007,TV-MA,104 min,"[dramas, international movies]",When he causes a car accident that leaves a yo...
5765,Movie,Splatter,Joe Dante,"[Corey Feldman, Tony Todd, Tara Leigh, Erin...",[United States],2009,2009,TV-MA,29 min,[horror movies],"After committing suicide, a washed-up rocker r..."
3840,Movie,Mad Ron's Prevues from Hell,Jim Monaco,"[Nick Pawlow, Jordu Schell, Jay Kushwara, M...",[United States],2010,1987,NR,84 min,"[cult movies, horror movies]","This collection cherry-picks trailers, forgott..."


In [17]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

In [19]:
types = df['type'].value_counts().reset_index()
types

,index,type
0,Movie,5377
1,TV Show,2400


In [26]:
trace = go.Pie(labels=types['index'], values=types['type'],pull=[0.1, 0],marker=dict(colors=["#fed049", "#007580"]),title="Netflix Content Types")
trace

Pie({
    'labels': array(['Movie', 'TV Show'], dtype=object),
    'marker': {'colors': ['#fed049', '#007580']},
    'pull': [0.1, 0],
    'title': {'text': 'Netflix Content Types'},
    'values': array([5377, 2400])
})

In [27]:
fig = go.Figure([trace])
fig.show()

In [29]:
 def get_categories(data):
      categories = {}
      for listed_in in data['listed_in']:
          for category in listed_in:
              category = category.lower().strip()
              if category in categories: # increase current category count
                categories[category] = categories[category] + 1
              else: # create new category in categories object
                categories[category] = 1
      return pd.DataFrame(categories.values(), index= categories.keys())

In [32]:
categories = get_categories(df).reset_index()
categories.columns = ["category", "count"]
categories

,category,count
0,dramas,2809
1,independent movies,673
2,thrillers,541
3,stand-up comedy,381
4,talk shows,52
5,international movies,2437
6,horror movies,312
7,cult movies,59
8,children,532
9,family movies,532


In [33]:
sorted_category=  categories.sort_values(by="count")
trace = go.Bar(x=sorted_category['count'], y=sorted_category['category'], orientation="h", 
               marker_color='MediumPurple')
layout = go.Layout(title="Countries with most content", height=700, 
                   legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure([trace], layout=layout)
fig.show()

In [34]:
def get_country_code(country):
    country_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
    if country == "South Korea":
        return "KOR"
    elif country == "Notrh Korea":
        return "PRK"
    elif country == "West Germany" or country == "East Germany":
        return "DEU"
    elif country == "Bahamas":
        return "BHM"
    elif country == "Soviet Union":
        return "RUS"
    else:
        try:
            return country_codes[country_codes["COUNTRY"] == country].reset_index()["CODE"][0]
        except:
            return None
        

def get_countries(data):
    countries = {}
    for cs in data['country']:
        for country in cs:
            if country == "other":
                continue
            country = country.strip()
            if country in countries: # increase current country count
                countries[country][0] = countries[country][0] + 1
            else: # create new country in countries object
                if get_country_code(country) is not None:
                    countries[country] = [1, get_country_code(country)]
    return pd.DataFrame(countries.values(), index= countries.keys())

countries = get_countries(df).reset_index()
countries.columns = ["country", "count", "code"]

In [35]:
fig = px.choropleth(countries, 
                    locations="code",
                    color="count",
                    hover_name="country",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()